In [1]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from qdrant_client import QdrantClient
from llama_index.core.retrievers import VectorIndexRetriever
import re

In [4]:
# ---------- CONNECT TO QDRANT ----------
client = QdrantClient(url="http://localhost:6333")

In [5]:
# ---------- CONFIG ----------
COLLECTION = "saudi_labor_law"

In [6]:
vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [7]:
# ---------- LOAD EXISTING INDEX ----------
embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base")
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    storage_context=storage_context,
    embed_model=embed_model
)

In [15]:
# ---------- CREATE RETRIEVER ----------
retriever = VectorIndexRetriever(index=index, similarity_top_k=5)

In [16]:
def detect_lang(text):
    return "ar" if re.search(r'[\u0600-\u06FF]', text) else "en"

def answer_query_auto(query: str):
    lang = detect_lang(query)
    results = retriever.retrieve(query)

    print(f"\n🔍 Query ({'Arabic' if lang=='ar' else 'English'}): {query}")
    for rank, r in enumerate(results, start=1):
        md = r.node.metadata

        print(f"\n--- Result {rank} | {md.get('article_name', '')}")
        if lang == "ar":
            print(f"📘 {md.get('arabic_name')} | {md.get('chapter_title_ar')} |  {md.get('part_title_ar')}")
            print(f"AR:  {md.get('arabic_content')}")
            print(f"EN: {md.get('english_content')[:120]}...\n")
        else:
            print(f"📘 {md.get('arabic_name')} | {md.get('chapter_title_ar')} |  {md.get('part_title_ar')}")
            print(f"AR:  {md.get('arabic_content')}")
            print(f"EN: {md.get('english_content')}...\n")

In [17]:
answer_query_auto("What does Article 23 say?")


🔍 Query (English): What does Article 23 say?

--- Result 1 | 
📘 المادة التاسعة والعشرون بعد المائتين مكرر | بدون فصل |  العقوبات
AR:  96 

يعاقــب بغرامــة لا تقــل عــن )200,000( مــائتي ألــف ريــال ولا تزيــد علــى )500,000( خمســمائة ألــف ريــال؛ كــل مــن يخــالف أحكــام الفقــرة 
)1( من المادة )الثلاثين( من هذا النظام، وذلك دون إخلال بما تقضي به المادة )التاسعة والعشرون بعد المائتين( منه.
EN: 96. A fine of not less than 200,000 rials and not more than 500,000 rials shall be imposed on anyone who contravenes the provisions of paragraph (1) of article 30 of the present Regulations, without prejudice to the requirements of article XXIX....


--- Result 2 | 
📘 المادة الثانية والثلاثون | بدون فصل |  توظيف غير السعوديين
AR:  لا يجوز الإستقدام بقصد العمل إلا بعد موافقة الوزارة .
EN: Proceedings for the purpose of work may be made only with the approval of the Ministry....


--- Result 3 | 
📘 المادة الحادية والعشرون | الأحكام العامة |  التعريفات والأحكام العامة
AR:  على الوزير – في سبي

In [19]:
answer_query_auto(" المادة العشرين ")


🔍 Query (Arabic):  المادة العشرين 

--- Result 1 | 
📘 المادة الخامسة والتسعون بعد المائة | بدون فصل |  تفتيش العمل
AR:  65 
)ملغاة(
EN: 65 (Cancelled)...


--- Result 2 | 
📘 المادة السابعة والتسعون بعد المائة | بدون فصل |  تفتيش العمل
AR:  67  
)ملغاة(
EN: 67 (Cancelled)...


--- Result 3 | 
📘 المادة الحادية والأربعون بعد المائتين | بدون فصل |  العقوبات
AR:  108 

)ملــــــغاة(
EN: 108 (Cancelled)...


--- Result 4 | 
📘 المادة الحادية والعشرون | الأحكام العامة |  التعريفات والأحكام العامة
AR:  على الوزير – في سبيل تنفيذ أحكام هذا النظام – التنسيق مع الجهات ذات العلاقة كلما اقتضى الأمر ذلك. 
 
 
 
 
 
 
 
 
 
 
 

 

 

 
6 

 
 
 

 
 

 

02
EN: In order to implement the provisions of this regulation, the Minister shall coordinate with the relevant authorities whe...


--- Result 5 | 
📘 المادة التاسعة والثلاثين بعد المائتين | بدون فصل |  العقوبات
AR:  106 

)ملــــــغاة(
EN: 106 (Cancelled)...



In [22]:
answer_query_auto("في 5 سنوات كم يستحق العامل رصيد الإجازات")


🔍 Query (Arabic): في 5 سنوات كم يستحق العامل رصيد الإجازات

--- Result 1 | 
📘 المادة الحادية عشرة بعد المائة | الإجازات |  شروط العمل وظروفه
AR:  للعامــل الحــق فــي الحصــول علــى أجــرة عــن أيــام الإجــازة المســتحقة إذا تــرك العمــل قبــل اســتعماله لهــا وذلــك بالنســبة إلــى المــدة 
التي لم يحصل على إجازته عنها كما يستحق أجرة الإجازة عن أجزاء السنة بنسبة ما قضاه منها في العمل.
EN: A worker has the right to be paid for days of qualifying leave if he leaves the job before he uses it, for as long as he...


--- Result 2 | 
📘 المادة الثانية عشرة بعد المائة | الإجازات |  شروط العمل وظروفه
AR:  لكل عامل الحق في إجازة بأجر كامل في الأعياد والمناسبات التي تحددها اللائحة .
EN: Every worker has the right to leave with full pay during the holidays and events specified in the regulation....


--- Result 3 | 
📘 المادة التاسعة بعد المائة | الإجازات |  شروط العمل وظروفه
AR:  1. يســتحق العامــل عــن كــل عــام إجــازة ســنوية لا تقــل مــدتها عــن واحــد وعشــرين يومــاً ، تــُزاد إلــى م

In [14]:
answer_query_auto(" أجر اضافي")


🔍 Query (Arabic):  أجر اضافي

--- Result 1 | 
📘 المادة السابعة بعد المائة | فترات الراحة والراحة الأسبوعية |  شروط العمل وظروفه
AR:  48 

1. يجــب علــى صــاحب العمــل أن يــدفع للعامــل أجــراً إضــافيًّا عــن ســاعات العمــل الإضــافية يــوازي أجــر الســاعة مضــافاً إليــه 

)50%( مــن أجــره الأساســي، ويجــوز لصــاحب العمــل بموافقــة العامــل أن يحتســب للعامــل أيــام إجــازة تعويضــية مدفوعــة 
الأجر بدلاً عن الأجر المستحق للعامل لساعات العمل الإضافية. وتبين اللائحة الأحكام المتصلة بذلك. 

2. إذا كــان التشــغيل فــي المنشــأة علــى أســاس المعيــار الأســبوعي لســاعات العمــل تعــد الســاعات التــي تزيــد علــى الســاعات 

المتخذة لهذا المعيار ساعات عمل إضافية . 

3. تعد جميع ساعات العمل التي تؤدَّى في أيام العطل والأعياد ساعات إضافية .
EN: 48.1 The employer must pay the worker an additional fee for overtime hours in addition to the hourly rate (50 per cent) ...


--- Result 2 | 
📘 المادة الثانية والثلاثون بعد المائتين | بدون فصل |  العقوبات
AR:  99 

يــتم تحصــيل الغرامـــا